<a href="https://colab.research.google.com/github/danielscurlock/DS12U1-BUILD/blob/master/notebooks/Explanatory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Explanatory Analysis of 188 Million Wild Fires:**
by: ***Daniel Scurlock*** (3/2/20)


---



# *Dataset Description:*
https://www.kaggle.com/rtatman/188-million-us-wildfires

This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24-year period.


---


***I will try to be as thorough as possible. My goal here is to ask questions and exmplain those answers with visulalizations***

In [0]:
!pip install geopandas shapely

In [0]:
# Get things ready for analysis
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes 
from shapely.geometry import Point, Polygon

In [0]:
# Get the previously created CSV into place for tomorrow
df = pd.read_csv('/content/drive/My Drive/Lambda/DSU1-BUILD/data/firesfinal.csv', index_col=0 )

# **A regional analysis of wildfires**
I will compare and contrast wildfire data based on region. I want to know the frequency of fires per region, and the average acreage burned. I want to identify the most frequest cause of of those fires per region. I also want to identify the months where fires are more frequent